In [ ]:
import pandas as pd
import json 

In [ ]:
data = []
with open("snli_1.0/snli_1.0_train.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df[df['gold_label'] == 'contradiction'].sample(10)[['sentence1', 'sentence2']]


In [ ]:
def convert_to_format(row):
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    prompt = """Below is an instruction that describes a task paired with input that provides further context. Write a response that appropriately completes the request."""
    instruction = """Given the following sentence, your job is to generate the negation for it in the json format"""
    input = str(sentence1)
    response = f"""```json
{{'orignal_sentence': '{sentence1}', 'generated_negation': '{sentence2}'}}
```
"""
    if len(input.strip()) == 0:  #  prompt + 2 new lines + ###instruction + new line + input + new line + ###response
        text = prompt + "\n\n### Instruction:\n" + instruction + "\n### Response:\n" + response
    else:
        text = prompt + "\n\n### Instruction:\n" + instruction + "\n### Input:\n" + input + "\n" + "\n### Response:\n" + response
    
    # we need 4 columns for auto train, instruction, input, output, text
    return pd.Series([instruction, input, response, text])

In [ ]:
new_df = df[df['gold_label'] == 'contradiction'][['sentence1', 'sentence2']].apply(convert_to_format, axis=1)
new_df.columns = ['instruction', 'input', 'output', 'text']

new_df.to_csv('snli_instruct.csv', index=False)